# Sheet 07

In [11]:
from numpy.typing import NDArray  # trying to be typesafe
import numpy as np  # needed all over the place
import scipy
from scipy.stats import skew
from IPython.display import HTML # in line animations

import os   # file and file path

from itertools import count
import pandas as pd
import matplotlib.pyplot as plt  # for plots

from matplotlib import colors  # not quite sure what fore
from matplotlib.ticker import PercentFormatter  # also not sure, maybe animation..?
from matplotlib.animation import FuncAnimation, FFMpegWriter  # for animations

##### Markov Chain - Monte Carlo  - Metropolis importance Sampling

In [ ]:
def initiate_positions_on_grid(
            num_particles: int,
            box: tuple [float, float],

    ) -> np.ndarray:
    """
    Initializes 2D positions on a Grid with even spacing.

    Parameters:
        
        n_particles:(int)
                -> number of particles
        
        box:(tuple[number,number]) 
                -> box size (in [nm])
        
    Return:
        (NDArray[float, shpae=(dimension, number of particles)])
                -> list of vectors of positons that are aranged on a grid
    """
    grid_sections = int(np.ceil(np.sqrt(num_particles)))  # find the number of colums & rows
    # even spacing
    x_spacing = box[0]/grid_sections 
    y_spacing = box[1]/grid_sections
    print("x_spacing:",x_spacing)
    print("y_spacing:",y_spacing)
    # makes grid coordinates
    x, y= np.meshgrid(
        np.arange(grid_sections) * x_spacing, 
        np.arange(grid_sections) * y_spacing
    )        # constants
    x= x.flatten()[:num_particles]+ x_spacing/2
    y= y.flatten()[:num_particles]+ y_spacing/2
    positions= np.linalg.matrix_transpose(np.array([x,y]))
    print("init positions: type",type(positions),"size",np.shape(positions))  
    return positions 

In [ ]:
def relative_distnace(
        x_1: np.ndarray, 
        x_2: np.ndarray, 
        box: tuple
):
    """
    Determines the minimum relative distance over the boundry conditions

    Parameters:
     
        x_1: (NDArray)
        position Vector of Particle 1

        x_2: (NDArray)
        position Vector of Particle 2
        
        box: (tuple[float, float])
        simulation box side length [nm] 

    Returns: 
     
        rel_vector: (NDArray)
                -> vector; minimal image convention
        
        distance: (float)
                -> flaot; minimal image convention
    """
    rel_vector = x_1 - x_2
    for dim in range(len(box)):
        rel_vector[dim] -= box[dim] * np.round(rel_vector[dim] / box[dim])
    return rel_vector 

In [ ]:
def initiate_distances(
        positions: np.ndarray,
        box: tuple
) -> NDArray:
    """retunrs matrix of distances (vector, scalar).

    Parameters:
        positions: (NDArray)
            -> initial positions; array of vectors

        box: (tuple)
            -> box edge lengths (x_length, y_length)
    
    Return: (tuple)
        vec_distances: (NDArray)
            -> (n x n x dim)-matrix (n= number of particles); relative distance i -> j with periodic boundry conditions and minimum image convention
        
        distances: (NDArray)
            -> (n x n)-matrix; delative distnace, norm value of vec_distances
    """
    vec_distance= np.zeros((len(positions),len(positions),2))
    print("shape of return[1]: rel_vect ", np.shape(vec_distance))
    for i in range(len(positions)):
        for j in range(i,len(positions)):
            vec_distance[i,j,:]= relative_distnace(positions[i],positions[j], box=box)
            vec_distance[j,i,:]= -vec_distance[i,j,:]
    return vec_distance


if __name__ == '__main__': 
    num_par=25
    box= (10,10)
    pos= initiate_positions_on_grid(num_par,box)
    d_vec= initiate_distances(pos, box)

    dx=np.array([0.5,0.5])
    print("self distance",d_vec[1,1])
    print("\n\nprior:\nd_vec",d_vec[1,10])
    d_vec[1]+=+dx
    pos[1]+=dx
    a_pos = relative_distnace(pos[1],pos[10],box)
    print("\n\nmatrix new distance",d_vec[1,10], np.linalg.norm(d_vec[1,10]))
    print("vector calculation", a_pos, np.linalg.vector_norm(a_pos))
    print("self distance",d_vec[1,1])

x_spacing: 2.0
y_spacing: 2.0
init positions: type <class 'numpy.ndarray'> size (25, 2)
shape of return[1]: rel_vect  (25, 25, 2)
self distance [-0. -0.]


prior:
d_vec [ 2. -4.]


matrix new distance [ 2.5 -3.5] 4.301162633521313
vector calculation [ 2.5 -3.5] 4.301162633521313
self distance [0.5 0.5]


In [ ]:
def montecarlo_step(shape:tuple, step_size:float) -> np.ndarray:
    """ Determines a random direction to take a step of step_size.
    
    Parameters: 

    Return:    
    """
    dx= np.random.rand(shape)
    dx= dx / np.linalg.norm(dx,axis=0) * step_size
    return dx

In [ ]:
def LJ_potential(r):
    """Lennard-Jones interaction potential
    """
    return 9.847044e-3/r**12-6.2647225/r**6  # [J]

In [ ]:
def metropolis_importance(
        vec_distances: np.ndarray,
        dx: np.ndarray
) -> NDArray:
    K_B = 1.38e-23  # [J/K]
    T= 293.15  # [K]
    distance1= np.linalg.vector_norm(vec_distances)
    vec_distances2= vec_distances+dx
    distance2= np.linalg.vector_norm(vec_distances2)
    e_1, e_2= LJ_potential(distance1), LJ_potential(distance2)  # [J]
    vec_distances=  np.where(
        e_1 >= e_2 or  -(e_2-e_1)/(K_B*T) >= np.log(np.random.uniform(0,1)) , #  if condition: e_1>=e_2 or ln P >= ln q
        vec_distances2,  #  True 
        vec_distances    # False
        )      
    return vec_distances

In [ ]:
def mcmi(num_par: int, rel_vec: np.ndarray, dx:float):
    K_B=1.38e23
    T= 293.15
    ab=np.zeros((num_par,num_par))
    for A in range(num_par):
        for B in range(A+1,num_par):
            dx = montecarlo_step(*np.shape(rel_vec[A,B,:]),step_size=dx)

            d1 = np.linalg.norm(rel_vec[A,B])
            d2 = np.linalg.norm(rel_vec[A,B]+dx)
            e_1, e_2= LJ_potential(d1), LJ_potential(d2)  # [J]

            if e_1 >= e_2 or  -(e_2-e_1)/(K_B*T) >= np.log(np.random.uniform(0,1)):
                result1=rel_vec[A,B]+dx 
                print("True")
            else:
                result1= rel_vec[A,B]
                print("False")
            ab[A,B]=result1
            ab[B,A]=-result1
            result2= metropolis_importance(vec_distances=rel_vec[A,B],dx=dx)

            print("result1-result2",result1-result2)

In [ ]:
NUM_PAR=5
BOX=(10,10)  # [nm]
DX=0.01  # [nm]
K_B= 1.38e-23
T=293.15
init_pos= initiate_positions_on_grid(NUM_PAR, BOX)
rel_vec= initiate_distances(positions=init_pos, box=BOX)

# consider rel_vec 1 -> 2

ab = np.zeros((NUM_PAR,NUM_PAR,2))
for A in range(NUM_PAR):
    for B in range(A+1,NUM_PAR):
        dx = montecarlo_step(*np.shape(rel_vec[A,B,:]),step_size=DX)

        d1 = np.linalg.norm(rel_vec[A,B])
        d2 = np.linalg.norm(rel_vec[A,B]+dx)
        e_1, e_2= LJ_potential(d1), LJ_potential(d2)  # [J]

        if e_1 >= e_2 or  -(e_2-e_1)/(K_B*T) >= np.log(np.random.uniform(0,1)):
            result1=rel_vec[A,B]+dx 
            print("True")
        else:
            result1= rel_vec[A,B]
            print("False")
        ab[A,B]=result1
        ab[B,A]=-result1
        result2= metropolis_importance(vec_distances=rel_vec[A,B],dx=dx)

        print("result1-result2",result1-result2)

restult3= metropolis_importance(rel_vec,dx)
print(ab-restult3)

x_spacing: 3.3333333333333335
y_spacing: 3.3333333333333335
init positions: type <class 'numpy.ndarray'> size (5, 2)
shape of return[1]: rel_vect  (5, 5, 2)
True
result1-result2 [0. 0.]
False
result1-result2 [0. 0.]
True
result1-result2 [0. 0.]
True
result1-result2 [0. 0.]
True
result1-result2 [0. 0.]
True
result1-result2 [0. 0.]
True
result1-result2 [0. 0.]
True
result1-result2 [0. 0.]
False
result1-result2 [0. 0.]
True
result1-result2 [0. 0.]
[[[ 0.          0.        ]
  [ 0.00932873  0.00360205]
  [ 0.          0.        ]
  [ 0.00455876  0.00890043]
  [ 0.00790539  0.00612411]]

 [[-0.00932873 -0.00360205]
  [ 0.          0.        ]
  [ 0.00807564  0.0058978 ]
  [ 0.00393308  0.00919407]
  [ 0.00290305  0.00956934]]

 [[ 0.          0.        ]
  [-0.00807564 -0.0058978 ]
  [ 0.          0.        ]
  [ 0.00991296  0.00131653]
  [ 0.          0.        ]]

 [[-0.00455876 -0.00890043]
  [-0.00393308 -0.00919407]
  [-0.00991296 -0.00131653]
  [ 0.          0.        ]
  [ 0.0011098